<a href="https://colab.research.google.com/github/golfn123/CE888Project/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticke
from tensorflow import keras



In [2]:
df = pd.read_json('https://raw.githubusercontent.com/golfn123/CE888Project/main/seq2seq/train_spider.json') #import dataset into dataframe
df.head

<bound method NDFrame.head of                       db_id  \
0     department_management   
1     department_management   
2     department_management   
3     department_management   
4     department_management   
...                     ...   
6995        culture_company   
6996        culture_company   
6997        culture_company   
6998        culture_company   
6999        culture_company   

                                                  query  \
0            SELECT count(*) FROM head WHERE age  >  56   
1     SELECT name ,  born_state ,  age FROM head ORD...   
2     SELECT creation ,  name ,  budget_in_billions ...   
3     SELECT max(budget_in_billions) ,  min(budget_i...   
4     SELECT avg(num_employees) FROM department WHER...   
...                                                 ...   
6995  SELECT T1.company_name FROM culture_company AS...   
6996  SELECT T1.title ,  T3.book_title FROM movie AS...   
6997  SELECT T1.title ,  T3.book_title FROM movie AS...   
6998  S

In [3]:
#check is there any missing value in dataframe
df.isnull().values.any()
#the value shows that the dataset has no missing value.

False

In [4]:
input = df['question']  #assign column question as input for learning model.
input

0       How many heads of the departments are older th...
1       List the name, born state and age of the heads...
2       List the creation year, name and budget of eac...
3       What are the maximum and minimum budget of the...
4       What is the average number of employees of the...
                              ...                        
6995    What are all the company names that have a boo...
6996    Show the movie titles and book titles for all ...
6997    What are the titles of movies and books corres...
6998    Show all company names with a movie directed i...
6999    What are all company names that have a corresp...
Name: question, Length: 7000, dtype: object

In [5]:
output = df['sql'] #assign sql column as an output for learning model.
output

0       {'from': {'table_units': [['table_unit', 1]], ...
1       {'from': {'table_units': [['table_unit', 1]], ...
2       {'from': {'table_units': [['table_unit', 0]], ...
3       {'from': {'table_units': [['table_unit', 0]], ...
4       {'from': {'table_units': [['table_unit', 0]], ...
                              ...                        
6995    {'from': {'table_units': [['table_unit', 2], [...
6996    {'from': {'table_units': [['table_unit', 1], [...
6997    {'from': {'table_units': [['table_unit', 1], [...
6998    {'from': {'table_units': [['table_unit', 1], [...
6999    {'from': {'table_units': [['table_unit', 1], [...
Name: sql, Length: 7000, dtype: object

In [6]:
df.columns #shows columns in in spider.json file

Index(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question',
       'question_toks', 'sql'],
      dtype='object')

In [7]:
#convert input into list
input_list = input.tolist()
input_list

['How many heads of the departments are older than 56 ?',
 'List the name, born state and age of the heads of departments ordered by age.',
 'List the creation year, name and budget of each department.',
 'What are the maximum and minimum budget of the departments?',
 'What is the average number of employees of the departments whose rank is between 10 and 15?',
 'What are the names of the heads who are born outside the California state?',
 "What are the distinct creation years of the departments managed by a secretary born in state 'Alabama'?",
 'What are the names of the states where at least 3 heads were born?',
 'In which year were most departments established?',
 "Show the name and number of employees for the departments managed by heads whose temporary acting value is 'Yes'?",
 'How many acting statuses are there?',
 'How many departments are led by heads who are not mentioned?',
 'What are the distinct ages of the heads who are acting?',
 "List the states where both the secretary

In [8]:
#convert output into list
output_list = output.tolist()
output_list

[{'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': [],
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'union': None,
  'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]]},
 {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': ['asc', [[0, [0, 10, False], None]]],
  'select': [False,
   [[0, [0, [0, 8, False], None]],
    [0, [0, [0, 9, False], None]],
    [0, [0, [0, 10, False], None]]]],
  'union': None,
  'where': []},
 {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 0]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': [],
  'select': [False,
   [[0, [0, [0, 3, False], None]],
    [0, [0, [0, 2, False], None]],
    [0, [0, [0, 5, False], None]]]],
  'union': None,
  'where': []},
 {'excep

In [9]:
output_list[6999]

{'except': None,
 'from': {'conds': [[False,
    2,
    [0, [0, 8, False], None],
    [0, 19, False],
    None]],
  'table_units': [['table_unit', 1], ['table_unit', 2]]},
 'groupBy': [],
 'having': [],
 'intersect': None,
 'limit': None,
 'orderBy': [],
 'select': [False, [[0, [0, [0, 14, False], None]]]],
 'union': None,
 'where': [[False, 2, [0, [0, 10, False], None], 1999.0, None]]}

In [32]:
# Vectorize the data.
input_texts = input_list
target_texts = output_list
input_characters = set()
target_characters = set()
for i in range(999):          #select only thousand samples for training
  input_text = input_texts[i]
  target_text = target_texts[i]
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 7000
Number of unique input tokens: 80
Number of unique output tokens: 10
Max sequence length for inputs: 224
Max sequence length for outputs: 10


KeyError: ignored

In [11]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
#num_samples = 5000  # Number of samples to train on.

In [26]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split = 0.2,
)

Epoch 1/100
88/88 [==============================] - 12s 91ms/step - loss: 0.4474 - accuracy: 0.0219 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
88/88 [==============================] - 7s 80ms/step - loss: 0.4465 - accuracy: 0.0219 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/100
88/88 [==============================] - 7s 80ms/step - loss: 0.4458 - accuracy: 0.0219 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/100
88/88 [==============================] - 7s 80ms/step - loss: 0.4452 - accuracy: 0.0219 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/100
88/88 [==============================] - 7s 80ms/step - loss: 0.4443 - accuracy: 0.0219 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/100
41/88 [============>.................] - ETA: 3s - loss: 0.4414 - accuracy: 0.0218

KeyboardInterrupt: ignored